In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime

In [16]:
def resample_to_constant_volume(df, volume_per_candle):
    # sort by time
    #df = df.sort_values(by='time')
    # create cumulative volume column
    df['cumulative_volume'] = df['volume'].cumsum()
    print(len(df))
    # initialize the new dataframe to hold resampled data
    resampled_df = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'])
    
    start_index = 0
    end_index = 0
    cumulative_volume = 0
    for i in range(len(df)):
        cumulative_volume += df.iloc[i]['volume']
        end_index = i
        if cumulative_volume >= volume_per_candle:
            candle_df = df.iloc[start_index:end_index+1]
            open_price = candle_df.iloc[0]['open']
            high_price = candle_df['high'].max()
            low_price = candle_df['low'].min()
            close_price = candle_df.iloc[-1]['close']
            mask = df.iloc[start_index]["date"].split("+")[0]
            date_open =  pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
            mask = df.iloc[end_index]["date"].split("+")[0]
            date_end = pd.to_datetime(datetime.strptime(mask, '%Y-%m-%d %H:%M:%S'))
       
            
            
            resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 
                                                'close': close_price, 'volume': cumulative_volume,
                                                "date_open": date_open, #"time_open":time_open,
                                                "date_end": date_end} #"time_end":time_end}
                                                 ,ignore_index=True)
            cumulative_volume = 0
            start_index = end_index + 1
    
    if start_index < len(df):
        candle_df = df.iloc[start_index:len(df)]
        open_price = candle_df.iloc[0]['open']
        high_price = candle_df['high'].max()
        low_price = candle_df['low'].min()
        close_price = candle_df.iloc[-1]['close']
        resampled_df = resampled_df.append({'open': open_price, 'high': high_price, 'low': low_price, 'close': close_price, 'volume': volume_per_candle}, ignore_index=True)

    return resampled_df

In [4]:
df_5min = pd.read_csv('ACC_5min-2.csv')

In [5]:
df_5min.volume.describe()

count    6.094500e+04
mean     1.195023e+04
std      2.249710e+04
min      1.900000e+01
25%      3.315000e+03
50%      6.845000e+03
75%      1.365000e+04
max      1.254536e+06
Name: volume, dtype: float64

In [6]:
df_total = pd.read_csv('ACC_minute_data_with_indicators.csv')

In [8]:
df_1min_all = df_total[['date' , 'close','high' , 'low' , 'open' , 'volume']]

In [9]:
df_18_21_1min = df_1min_all.iloc[249971:554674]

In [10]:
df_18_21_1min

,date,close,high,low,open,volume
249971,2018-01-01 09:15:00+05:30,1760.00,1763.90,1758.65,1763.90,718
249972,2018-01-01 09:16:00+05:30,1763.80,1763.80,1760.00,1760.00,409
249973,2018-01-01 09:17:00+05:30,1760.10,1763.45,1760.00,1763.45,867
249974,2018-01-01 09:18:00+05:30,1759.35,1760.45,1759.35,1760.10,1065
249975,2018-01-01 09:19:00+05:30,1758.65,1759.90,1758.65,1759.35,436
...,...,...,...,...,...,...
554669,2021-07-28 15:24:00+05:30,2395.35,2395.65,2394.05,2395.65,1576
554670,2021-07-28 15:25:00+05:30,2391.70,2396.75,2390.15,2395.35,2622
554671,2021-07-28 15:26:00+05:30,2395.00,2395.35,2391.70,2391.70,2092
554672,2021-07-28 15:27:00+05:30,2391.15,2395.00,2391.15,2394.95,1189


In [13]:
# 4 times the 90% of 1 min data volume: 1 volume candle consist of 4 x 1 min candle 
vol_bar = df_18_21_1min.volume.quantile(0.90)*4
vol_bar

21240.0

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_eq_vol = resample_to_constant_volume(df_18_21_1min, vol_bar)

304703


In [21]:
df

NameError: name 'df' is not defined

In [32]:
df_18_21_1min.iloc[249972][['open' , 'high']]

open     1670.8
high    1672.45
Name: 499943, dtype: object

In [33]:
df_18_21_1min.iloc[249972][['open' , 'high']].shift()

open       NaN
high    1670.8
Name: 499943, dtype: object